In [1]:
emotion = "fear"
emotion = "sadness"
emotion = "joy"

emotion = "anger"

In [2]:
data_folder = "E:\data\emoint"
#data_file = "E:\\data\\emoint\\tweets_sm.xlsx"
data_file = "E:\\data\\emoint\\tweets_all.xlsx"

In [3]:
import os
import re
import csv
import time
import keras
import numpy
import random
import tensorflow

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers

In [5]:
from keras import *
from keras.utils import *
from keras.models import *
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import KFold

In [6]:
max_features = 20000
sequence_length = 100 # max number of words in one tweet 

df_list = []

for f in os.listdir(data_folder):
    if bool(re.search(r'\.txt$', f)):
        f1 = '{}\{}'.format(data_folder, f)
        df = pd.read_csv(f1, 
            header = 0,
            names = ['tweet_id', 'text', 'emotion', 'intensity'],
            delimiter = '\t',
            )
        df_list.append(df)

df_all = pd.concat(df_list)

#df_all = df_all.head(1000)

df_all.to_excel(
    '{}/tweets_all.xlsx'.format(data_folder),
    index = False,
    )


# data conversion functions

## function to convert text file to text and tag list

In [7]:
def convert_file_to_text_and_tag_list(
    emotion_tag,
    data_file,
    ):
    data = pd.read_excel(data_file)
    data['label'] = data['emotion'].apply(lambda x: 1 if x == emotion else 0)
    texts = data['text'].to_list()
    tags = data['label'].to_list()
    return texts, tags

## function to convert text file to text and intensentiy score list

In [8]:
def str_to_float(
    x):
    try:
        return float(x)
    except:
        return None
    
def convert_file_to_text_and_score_list(
    emotion_tag,
    data_file,
    ):
    data = pd.read_excel(data_file)
    data = data[data['emotion'] == emotion]
    data['intensity'] = data['intensity'].apply(str_to_float)
    data = data[data.intensity.notnull()]
    texts = data['text'].to_list()
    scores = data['intensity'].to_list()
    return texts, scores

# build the model

# parameters

## convert text list to a input format of deep learning model

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained(
    "bert-base-uncased", do_lower_case=True
    )

In [10]:
def texts_to_input(
    texts,
    sequence_length = 100,
    ):
    encoded = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=sequence_length,
        return_attention_mask=True,
        return_token_type_ids=True,
        pad_to_max_length=True,
        return_tensors="tf",
        )
    # Convert batch of encoded features to numpy array.
    input_ids = np.array(encoded["input_ids"], dtype="int32")
    attention_masks = np.array(encoded["attention_mask"], dtype="int32")
    token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
    return input_ids, attention_masks, token_type_ids

texts_to_input(["Never dull moment here"])

texts_to_input(["Never dull moment here"], method = 'one_hot')

## function of building the model of text emotion tag 

In [11]:
embedding_dim = 300
dropout_rate = 0.2
filters = 128

In [12]:
def emotion_tagger_model_building(
    embedding_dim = 300,
    filters = 128,
    kernel_size = 2,
    dropout_rate = 0.2,
    sequence_length = 100,
    ):
    ### input layers
    input_ids = tf.keras.layers.Input(
        shape=(sequence_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(sequence_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(sequence_length,), dtype=tf.int32, name="token_type_ids"
    )

    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False
    ###load the bert model 
    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state   
    x = layers.Dropout(dropout_rate)(sequence_output)
    # Conv1D + global max pooling
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    # We add a vanilla hidden layer:
    x = layers.Dense(filters, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(2, 
        activation="softmax",
         name="predictions")(x)
    model = keras.Model([
        input_ids,
        attention_masks,
        token_type_ids,
    ], predictions)
    model.compile(
        loss="categorical_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])
    return model

## function of train the model of emotion tagger with data

In [13]:
def train_tagger(
    texts,
    tags,
    tagger_model_path = None,
    tagger_model_weight_path = None,
    tagger_model_json_path = None,
    epochs = 10,
    validation_split = 0.1,
    dropout_rate = 0.2,
    ):
    tagger_model = emotion_tagger_model_building(
        dropout_rate = dropout_rate,
        )
    '''
    prepare the text input

    texts = [
        "i feel so fear",
        "nothing is wrong"
        ]
    '''
    x_ids, x_attention, x_type = texts_to_input(texts)
    '''
    prepare the output
    '''
    y = numpy.array(tags)
    y = to_categorical(y)
    print(x_ids.shape)
    print(x_attention.shape)
    print(x_type.shape)
    print(y.shape)
    print(numpy.sum(y, axis = 0))
    # Fit the model using the train and test datasets.
    tagger_model.fit(
        [x_ids, x_attention, x_type], y, 
        validation_split=validation_split, 
        epochs=epochs)
    if tagger_model_path is not None:
        tagger_model.save(tagger_model_path)
    return tagger_model

# function of intensity score model building

In [14]:
def emotion_scorer_model_building(
    embedding_dim = 300,
    filters = 128,
    kernel_size = 2,
    dropout_rate = 0.2,
    sequence_length = 100,
    ):   
    ### input layers
    input_ids = tf.keras.layers.Input(
        shape=(sequence_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(sequence_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(sequence_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False
    ###load the bert model 
    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state   
    x = layers.Dropout(dropout_rate)(sequence_output)
    # Conv1D + global max pooling
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.Conv1D(filters, kernel_size, padding="valid", activation="relu")(x)
    x = layers.GlobalMaxPooling1D()(x)
    # We add a vanilla hidden layer:
    x = layers.Dense(filters, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    # We project onto a single unit output layer, and squash it with a sigmoid:
    predictions = layers.Dense(1, 
        activation="sigmoid",
        name="predictions")(x)
    model = keras.Model([
        input_ids,
        attention_masks,
        token_type_ids,
    ], predictions)
    model.compile(
        loss="mse", 
        optimizer="adam", 
        metrics=[metrics.mean_absolute_error])
    return model

# function of training intensity score model training

In [15]:
def train_scorer(
    texts,
    scores,
    scorer_model_path,
    epochs = 10,
    validation_split=0.1,
    ):
    scorer_model = emotion_scorer_model_building()
    '''
    prepare the text input
    texts = [
        "i feel so fear",
        "nothing is wrong"
        ]
    '''
    x_ids, x_attention, x_type = texts_to_input(texts)
    '''
    prepare the output
    '''
    y = numpy.array(scores)
    print(x_ids.shape)
    print(x_attention.shape)
    print(x_type.shape)
    print(y.shape)
    # Fit the model using the train and test datasets.
    scorer_model.fit(
        [x_ids, x_attention, x_type], 
        y, 
        validation_split=0.1, 
        epochs=epochs)
    scorer_model.save(
        scorer_model_path,
        save_format='h5')
    return scorer_model

## training the model

### K-fold cross validation function of tagger

In [16]:
def tagger_model_fold_cross_validation(
    texts,
    tags,
    n_splits = 5,
    ):
    #convert data to numpy arrays
    x_ids, x_attention, x_type = texts_to_input(texts)
    y = numpy.array(tags)
    y = to_categorical(y)
    #make the k folds
    kfold = KFold(n_splits = n_splits, shuffle=True)
    #
    acc_per_fold = []
    loss_per_fold = []
    #
    fold_no = 1
    for train, test in kfold.split(x_ids, y):
        tagger_model = emotion_tagger_model_building()
        tagger_model.fit(
            [x_ids[train], x_attention[train], x_type[train]],
            y[train], 
            epochs = 5,
            verbose = 1)
        scores = tagger_model.evaluate(
            [x_ids[test], x_attention[test], x_type[test]],
            y[test], 
            verbose = 1)
        print('accuracy of the {}-th fold:{}'.format(
            fold_no,
            scores[1]))
        acc_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])
        # Increase fold number
        fold_no = fold_no + 1
    ###
    acc_10_fold_cross_validation = numpy.mean(numpy.array(acc_per_fold))
    print('accuracy of {}-fold cross validation:\t{}'.format(
        n_splits,
        acc_10_fold_cross_validation,
        ))

### K-fold cross validation function of scorer

In [17]:
def scorer_model_fold_cross_validation(
    texts,
    scores,
    n_splits = 5,
    ):
    #convert data to numpy arrays
    x_ids, x_attention, x_type = texts_to_input(texts)
    y = numpy.array(scores)
    #make the k folds
    kfold = KFold(n_splits = n_splits, shuffle=True)
    #
    acc_per_fold = []
    loss_per_fold = []
    #
    fold_no = 1
    for train, test in kfold.split(x_ids, y):
        tagger_model = emotion_scorer_model_building()
        tagger_model.fit(
            [x_ids[train], x_attention[train], x_type[train]],
            y[train], 
            epochs = 5,
            verbose = 1)
        scores = tagger_model.evaluate(
            [x_ids[test], x_attention[test], x_type[test]],
            y[test], 
            verbose = 1)
        print('mse of the {}-th fold:{}'.format(
            fold_no,
            scores[1]))
        acc_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])
        # Increase fold number
        fold_no = fold_no + 1
    ###
    acc_10_fold_cross_validation = numpy.mean(numpy.array(acc_per_fold))
    print('mse of {}-fold cross validation:\t{}'.format(
        n_splits,
        acc_10_fold_cross_validation,
        ))

# training of each emotion

### tagger training 

#### load data

In [18]:
fear_texts, fear_tags = convert_file_to_text_and_tag_list(
    emotion_tag = emotion,
    data_file = data_file)

#### 5-fold cross validation

In [19]:
tagger_model_fold_cross_validation(
    fear_texts,
    fear_tags,
    )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\users\gaoyu\appdata\local\programs\python\python39\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some layers from the model checkpoint at bert-base-uncased were not

Epoch 1/5
265/265 [==============================] - 449s 2s/step - loss: 0.4432 - accuracy: 0.8049
Epoch 2/5
265/265 [==============================] - 477s 2s/step - loss: 0.3025 - accuracy: 0.8744
Epoch 3/5
265/265 [==============================] - 408s 2s/step - loss: 0.2466 - accuracy: 0.9023
Epoch 4/5
265/265 [==============================] - 425s 2s/step - loss: 0.2065 - accuracy: 0.9176
Epoch 5/5
67/67 [==============================] - 116s 2s/step - loss: 0.1657 - accuracy: 0.9390
accuracy of the 1-th fold:0.9390071034431458


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/5
265/265 [==============================] - 445s 2s/step - loss: 0.4531 - accuracy: 0.8013
Epoch 2/5
265/265 [==============================] - 424s 2s/step - loss: 0.2975 - accuracy: 0.8786
Epoch 3/5
265/265 [==============================] - 428s 2s/step - loss: 0.2492 - accuracy: 0.9022
Epoch 4/5
265/265 [==============================] - 422s 2s/step - loss: 0.2148 - accuracy: 0.9163
Epoch 5/5
67/67 [==============================] - 127s 2s/step - loss: 0.1659 - accuracy: 0.9324
accuracy of the 2-th fold:0.9323557019233704


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/5
265/265 [==============================] - 580s 2s/step - loss: 0.4421 - accuracy: 0.8037
Epoch 2/5
265/265 [==============================] - 1942s 7s/step - loss: 0.2986 - accuracy: 0.8777
Epoch 3/5
265/265 [==============================] - 537s 2s/step - loss: 0.2483 - accuracy: 0.9008
Epoch 4/5
265/265 [==============================] - 580s 2s/step - loss: 0.2171 - accuracy: 0.9164
Epoch 5/5
67/67 [==============================] - 110s 2s/step - loss: 0.1629 - accuracy: 0.9390
accuracy of the 3-th fold:0.9389782547950745


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/5
265/265 [==============================] - 486s 2s/step - loss: 0.4552 - accuracy: 0.7925
Epoch 2/5
265/265 [==============================] - 510s 2s/step - loss: 0.3094 - accuracy: 0.8716
Epoch 3/5
265/265 [==============================] - 580s 2s/step - loss: 0.2466 - accuracy: 0.9001
Epoch 4/5
265/265 [==============================] - 570s 2s/step - loss: 0.2122 - accuracy: 0.9175
Epoch 5/5
67/67 [==============================] - 114s 2s/step - loss: 0.2056 - accuracy: 0.9257
accuracy of the 4-th fold:0.925733208656311


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/5
265/265 [==============================] - 450s 2s/step - loss: 0.4368 - accuracy: 0.8131
Epoch 2/5
265/265 [==============================] - 1039s 4s/step - loss: 0.2958 - accuracy: 0.8839
Epoch 3/5
265/265 [==============================] - 3477s 13s/step - loss: 0.2408 - accuracy: 0.9055
Epoch 4/5
265/265 [==============================] - 551s 2s/step - loss: 0.2165 - accuracy: 0.9149
Epoch 5/5
67/67 [==============================] - 111s 2s/step - loss: 0.1543 - accuracy: 0.9395
accuracy of the 5-th fold:0.939451277256012
accuracy of 5-fold cross validation:	0.9351051092147827


#### train single model

In [20]:
start_time = time.time()

fear_tagger = train_tagger(
    texts = fear_texts,
    tags = fear_tags,
    tagger_model_path = '/data/emoint/{}_tagger_bert.h5'.format(emotion),
    epochs = 10,
    validation_split = 0.1,
    )

end_time = time.time()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


(10571, 100)
(10571, 100)
(10571, 100)
(10571, 2)
[8031. 2540.]
Epoch 1/10
298/298 [==============================] - 637s 2s/step - loss: 0.4337 - accuracy: 0.8071 - val_loss: 0.2826 - val_accuracy: 0.8875
Epoch 2/10
298/298 [==============================] - 669s 2s/step - loss: 0.2888 - accuracy: 0.8824 - val_loss: 0.2391 - val_accuracy: 0.9149
Epoch 3/10
298/298 [==============================] - 539s 2s/step - loss: 0.2348 - accuracy: 0.9073 - val_loss: 0.2294 - val_accuracy: 0.9178
Epoch 4/10
298/298 [==============================] - 534s 2s/step - loss: 0.1940 - accuracy: 0.9211 - val_loss: 0.1923 - val_accuracy: 0.9329
Epoch 5/10
298/298 [==============================] - 700s 2s/step - loss: 0.1701 - accuracy: 0.9338 - val_loss: 0.2113 - val_accuracy: 0.9159
Epoch 6/10
298/298 [==============================] - 631s 2s/step - loss: 0.1453 - accuracy: 0.9454 - val_loss: 0.2365 - val_accuracy: 0.9130
Epoch 7/10
298/298 [==============================] - 568s 2s/step - loss: 0.1

In [21]:
print('training time of scorer:\t{}'.format(end_time - start_time))

training time of scorer:	5546.219535112381


### scorer training

#### laod data

In [22]:
fear_texts, fear_scores = convert_file_to_text_and_score_list(
    emotion_tag = emotion,
    data_file = data_file)

#### 5-fold cross validation

In [23]:
scorer_model_fold_cross_validation(
    fear_texts,
    fear_scores,
    )

c:\users\gaoyu\appdata\local\programs\python\python39\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint 

Epoch 1/5
43/43 [==============================] - 45s 799ms/step - loss: 0.0329 - mean_absolute_error: 0.1419
Epoch 2/5
43/43 [==============================] - 34s 796ms/step - loss: 0.0188 - mean_absolute_error: 0.1100
Epoch 3/5
43/43 [==============================] - 34s 795ms/step - loss: 0.0166 - mean_absolute_error: 0.1027
Epoch 4/5
43/43 [==============================] - 34s 798ms/step - loss: 0.0130 - mean_absolute_error: 0.0899
Epoch 5/5
11/11 [==============================] - 6s 275ms/step - loss: 0.0144 - mean_absolute_error: 0.0947
mse of the 1-th fold:0.0946657657623291


ResourceExhaustedError: Exception encountered when calling layer "intermediate" (type TFBertIntermediate).

failed to allocate memory [Op:Mul]

Call arguments received:
  • hidden_states=tf.Tensor(shape=(3, 5, 768), dtype=float32)

#### train single model

In [ ]:
start_time = time.time()

fear_tagger = train_scorer(
    texts = fear_texts,
    scores = fear_scores,
    scorer_model_path = '/data/emoint/{}_scorer_bert.h5'.format(emotion),
    epochs = 10,
    validation_split = 0.1,
    )

end_time = time.time()

In [ ]:
print('training time of scorer:\t{}'.format(end_time - start_time))